In [1]:
#!/usr/bin/env python
# coding: utf-8

In [2]:
from datetime import datetime as dt
import pandas as pd

In [3]:
prev = pd.read_csv("prev.csv", encoding='shift-jis', header=0)
curr = pd.read_csv("maps.csv", encoding='shift-jis', header=0)

In [4]:
n = len(curr) - len(prev)
today = dt.now().isoformat()[:10]

In [5]:
if n > 0:
    msg = ["{}、道内で {} 件の新規感染報告が発生しています。".format(today, n)]
else:
    msg = ["{}、北海道の新型コロナ感染者の報告状況です。".format(today)]
for k, v in curr.iterrows():
    s = ""
    if k < len(prev):
        diff = curr.at[k, 'patients'] - prev.at[k, 'patients']
        if diff > 0:
            s = "{}、{}、{}名[+{}名]".format(
                curr.at[k, 'state'], curr.at[k, 'name'],
                curr.at[k, 'patients'], diff)
    else:
        s = "{}、{}、{}名 ※新規".format(
            curr.at[k, 'state'], curr.at[k, 'name'], curr.at[k, 'patients'])
    if s != "":
        msg.append(s)

if len(msg) > 1:
    for m in sorted(msg):
        print(m)

2021-02-11、道内で 3 件の新規感染報告が発生しています。
余市町、養老ホーム かるな和順、8名[+2名]
函館市、函館少年刑務所(市8例)、22名[+2名]
小樽市、小樽協会病院(市13例)、56名[+1名]
恵庭市、恵庭第一病院、1名 ※新規
札幌市、ススキノ ラウンジ(市165例)、5名 ※新規
札幌市、札幌市新生児童会館、1名 ※新規
札幌市、障害福祉サービス事業所(市156例)、88名[+1名]
江別市、介護老人保健施設 夢あかり、26名[+1名]
